In [ ]:
# 基于 TensorFlow Keras 的股票价格预测模型的比较分析

# ================================
# 导入必要的库和模块
# ================================

# 标准库
import os  # 用于与操作系统交互
import json  # 用于处理 JSON 数据
import random  # 用于生成随机数
import math  # 用于数学运算，如计算平方根
import time  # 导入 time 模块

# 数据处理和可视化
import numpy as np  # 导入 NumPy 库，用于高效的数值计算
import pandas as pd  # 导入 Pandas 库，用于数据处理和分析
import matplotlib.pyplot as plt  # 导入 Matplotlib 库，用于绘制图形

# 深度学习和神经网络
import tensorflow as tf  # 导入 TensorFlow 库，用于深度学习任务
from tensorflow.keras.models import Sequential, load_model  # 导入 Keras 的 Sequential 模型和加载模型功能
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional  # 导入常用的 Keras 层
from tensorflow.keras.callbacks import EarlyStopping  # 导入早停回调

# 预处理和评估
from sklearn.preprocessing import MinMaxScaler  # 导入 MinMaxScaler，用于特征归一化
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score  # 导入评估指标
from sklearn.model_selection import train_test_split  # 导入 train_test_split，用于划分数据集

# 忽略不必要的警告
import warnings
warnings.filterwarnings("ignore")  # 忽略所有警告

# ================================
# 设置随机种子以确保实验可复现
# ================================

seed_num = 42  # 设置随机种子数
os.environ['PYTHONHASHSEED'] = str(seed_num)  # 设置 Python 的 hash 种子
random.seed(seed_num)  # 设置 Python 随机数种子
np.random.seed(seed_num)  # 设置 NumPy 随机数种子
tf.random.set_seed(seed_num)  # 设置 TensorFlow 随机种子

# ================================
# 设置早停回调
# ================================

earlystop = EarlyStopping(
    monitor='val_loss',  # 监控验证集的损失
    min_delta=0,  # 最小变化量，变化不足则认为没有改进
    patience=20,  # 如果验证集损失在 20 个 epoch 内没有改善，则停止训练
    verbose=2  # 输出训练过程的详细程度
)

# ================================
# 读取和预处理数据
# ================================

# 读取股票数据
data = pd.read_csv('IBM.csv') 
print(data.head())  # 输出前几行数据，检查数据格式

# 选择需要的特征列（排除日期列）
all_data = data.iloc[:, 1:7]  # 选择第1到第6列（不包含第0列日期）
print(all_data.head())  # 显示选择后的数据前几行

# 进一步选择特定的列
all_data = all_data.loc[:, ['Open', 'Close', 'High', 'Low', 'Volume', 'Adj Close']]  
print(all_data.info())  # 输出数据的基本信息
print(all_data.describe())  # 输出数据的统计信息

# 绘制每个特征的分布直方图
all_data.hist(bins=50, figsize=(20, 15))
plt.show()

# 特征归一化，将所有特征缩放到 [0, 1] 范围
sc = MinMaxScaler(feature_range=(0, 1))
all_data_scaled = sc.fit_transform(all_data)
print(all_data_scaled)  # 输出归一化后的数据
print('训练数据长度是:', len(all_data_scaled))  # 输出数据长度

# 设置时间步长为 60
features = []  # 存储特征
labels = []  # 存储标签
for i in range(60, len(all_data_scaled)):  # 从第60条数据开始
    features.append(all_data_scaled[i-60:i, ])  # 取前60个时间步作为特征
    labels.append(all_data_scaled[i, 1])  # 将第1列 'Close' 作为标签
features, labels = np.array(features), np.array(labels)  # 转换为 NumPy 数组
features = np.reshape(features, (features.shape[0], features.shape[1], -1))  # 调整特征数组形状，适应 LSTM 输入

# 划分数据集为训练集、验证集和测试集
x_train, x_val, x_test = features[:6550], features[6550:6914], features[6914:]
y_train, y_val, y_test = labels[:6550], labels[6550:6914], labels[6914:]

# 打印数据集的形状
print('shape of x_train:', x_train.shape)
print('shape of x_val:', x_val.shape)
print('shape of x_test:', x_test.shape)
print('shape of y_train:', y_train.shape)
print('shape of y_val:', y_val.shape)
print('shape of y_test:', y_test.shape)

# ================================
# 构建和训练 LSTM 模型
# ================================
lstart_time = time.time()  # 记录开始时间
# 初始化 LSTM 模型
regressor = Sequential()

# 添加第一层 LSTM 和 Dropout
regressor.add(LSTM(
    units=50, 
    return_sequences=True, 
    input_shape=(x_train.shape[1], 6)  # 输入形状为 (时间步长, 特征数)
))
regressor.add(Dropout(0.2))  # 防止过拟合，丢弃 20% 的神经元

# 添加第二层 LSTM 和 Dropout
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

# 添加第三层 LSTM 和 Dropout
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

# 添加第四层 LSTM 和 Dropout
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

# 添加输出层
regressor.add(Dense(units=1))  # 输出一个值：预测的股票价格

# 编译 LSTM 模型
regressor.compile(optimizer='adam', loss='mean_squared_error')
print(regressor.summary())  # 输出模型结构摘要

# 训练 LSTM 模型
history = regressor.fit(
    x_train, y_train, 
    epochs=25,  # 训练 epoch 次数 分别测试了 5 / 15 /25
    batch_size=32,  # 每批次 32 个样本
    validation_data=(x_val, y_val), 
    callbacks=[earlystop]  # 仅使用早停回调
)

lend_time = time.time()  # 记录结束时间
lstm_runtime = lend_time - lstart_time


# 绘制 LSTM 的训练损失和验证损失曲线
loss = history.history['loss']  # 训练损失
val_loss = history.history['val_loss']  # 验证损失
epochs_range = range(1, len(loss) + 1)

plt.figure(figsize=(5, 3))
plt.title('LSTM Loss Curve')
plt.plot(epochs_range, loss, 'red', label='Training Loss')
plt.plot(epochs_range, val_loss, 'blue', label='Validation Loss')
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 归一化目标变量 'Close' 列
sc_one = MinMaxScaler(feature_range=(0, 1))
sc_one.fit(all_data.iloc[:, 1:2])  # 仅拟合，不进行转换

# 预测训练集和测试集的股票价格
predicted_stock_train = regressor.predict(x_train)
predicted_stock_train = sc_one.inverse_transform(predicted_stock_train)

predicted_stock_test_lstm = regressor.predict(x_test)
predicted_stock_test_lstm = sc_one.inverse_transform(predicted_stock_test_lstm)

# 获取真实的股票价格
real_price_train = sc_one.inverse_transform(np.reshape(y_train, (-1, 1)))
real_price_test = sc_one.inverse_transform(np.reshape(y_test, (-1, 1)))

# 可视化 LSTM 模型在训练集上的预测结果
plt.figure(figsize=(5, 3))
plt.plot(real_price_train, color='red', label='Real Stock Price')
plt.plot(predicted_stock_train, color='blue', label='Predicted Stock Price')
plt.title('LSTM Train Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.grid(True)
plt.show()

# 可视化 LSTM 模型在测试集上的预测结果
plt.figure(figsize=(5, 3))
plt.plot(real_price_test, color='red', label='Real Stock Price')
plt.plot(predicted_stock_test_lstm, color='blue', label='Predicted Stock Price')
plt.title('LSTM Test Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.grid(True)
plt.show()

# ================================
# 评估 LSTM 模型
# ================================

# 计算评估指标
lstm_mse = mean_squared_error(predicted_stock_test_lstm, real_price_test)  # 均方误差
lstm_rmse = math.sqrt(lstm_mse)  # 均方根误差
lstm_mae = mean_absolute_error(predicted_stock_test_lstm, real_price_test)  # 平均绝对误差
lstm_r2 = r2_score(predicted_stock_test_lstm, real_price_test)  # 决定系数

# 输出评估结果
print('LSTM 模型的均方误差 (MSE): %.6f' % lstm_mse)
print('LSTM 模型的均方根误差 (RMSE): %.6f' % lstm_rmse)
print('LSTM 模型的平均绝对误差 (MAE): %.6f' % lstm_mae)
print('LSTM 模型的决定系数 (R-squared): %.6f' % lstm_r2)
print('LSTM 模型运行时间: %.2f 秒' % lstm_runtime)

# ================================
# 构建和训练 GRU 模型
# ================================
gstart_time = time.time()  # 记录开始时间
# 初始化 GRU 模型
regressorGRU = Sequential()

# 添加第一层 GRU 和 Dropout
regressorGRU.add(GRU(
    units=50, 
    return_sequences=True, 
    input_shape=(x_train.shape[1], 6)
))
regressorGRU.add(Dropout(0.2))

# 添加第二层 GRU 和 Dropout
regressorGRU.add(GRU(units=50, return_sequences=True))
regressorGRU.add(Dropout(0.2))

# 添加第三层 GRU 和 Dropout
regressorGRU.add(GRU(units=50, return_sequences=True))
regressorGRU.add(Dropout(0.2))

# 添加第四层 GRU 和 Dropout
regressorGRU.add(GRU(units=50))
regressorGRU.add(Dropout(0.2))

# 添加输出层
regressorGRU.add(Dense(units=1))

# 编译 GRU 模型
regressorGRU.compile(optimizer='adam', loss='mean_squared_error')
print(regressorGRU.summary())

# 训练 GRU 模型
history = regressorGRU.fit(
    x_train, y_train, 
    epochs=25, 
    batch_size=32, 
    validation_data=(x_val, y_val), 
    callbacks=[earlystop]  # 仅使用早停回调
)

gend_time = time.time()  # 记录结束时间
gru_runtime = gend_time - gstart_time

# 绘制 GRU 的训练损失和验证损失曲线
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, len(loss) + 1)

plt.figure(figsize=(5, 3))
plt.title('GRU Loss Curve')
plt.plot(epochs_range, loss, 'red', label='Training Loss')
plt.plot(epochs_range, val_loss, 'blue', label='Validation Loss')
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 归一化目标变量 'Close' 列（重复操作，保留以确保功能一致）
sc_one = MinMaxScaler(feature_range=(0, 1))
sc_one.fit(all_data.iloc[:, 1:2])  # 仅拟合，不进行转换

# 预测训练集和测试集的股票价格
predicted_stock_train = regressorGRU.predict(x_train)
predicted_stock_train = sc_one.inverse_transform(predicted_stock_train)

predicted_stock_test_gru = regressorGRU.predict(x_test)
predicted_stock_test_gru = sc_one.inverse_transform(predicted_stock_test_gru)

# 获取真实的股票价格
real_price_train = sc_one.inverse_transform(np.reshape(y_train, (-1, 1)))
real_price_test = sc_one.inverse_transform(np.reshape(y_test, (-1, 1)))

# 可视化 GRU 模型在训练集上的预测结果
plt.figure(figsize=(5, 3))
plt.plot(real_price_train, color='red', label='Real Stock Price')
plt.plot(predicted_stock_train, color='blue', label='Predicted Stock Price')
plt.title('GRU Train Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.grid(True)
plt.legend()
plt.show()

# 可视化 GRU 模型在测试集上的预测结果
plt.figure(figsize=(5, 3))
plt.plot(real_price_test, color='red', label='Real Stock Price')
plt.plot(predicted_stock_test_gru, color='blue', label='Predicted Stock Price')
plt.title('GRU Test Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.grid(True)
plt.legend()
plt.show()

# ================================
# 评估 GRU 模型
# ================================

# 计算评估指标
gru_mse = mean_squared_error(predicted_stock_test_gru, real_price_test)
gru_rmse = math.sqrt(gru_mse)
gru_mae = mean_absolute_error(predicted_stock_test_gru, real_price_test)
gru_r2 = r2_score(predicted_stock_test_gru, real_price_test)

# 输出评估结果
print('GRU 模型的均方误差 (MSE): %.6f' % gru_mse)
print('GRU 模型的均方根误差 (RMSE): %.6f' % gru_rmse)
print('GRU 模型的平均绝对误差 (MAE): %.6f' % gru_mae)
print('GRU 模型的决定系数 (R-squared): %.6f' % gru_r2)
print('GRU 模型运行时间: %.2f 秒' % gru_runtime)

# ================================
# 构建和训练 BiLSTM 模型
# ================================
bistart_time = time.time()  # 记录开始时间
# 初始化 BiLSTM 模型
regressorBiLSTM = Sequential()

# 添加第一层双向 LSTM 和 Dropout
regressorBiLSTM.add(Bidirectional(
    LSTM(
        units=50, 
        return_sequences=True
    ), 
    input_shape=(x_train.shape[1], 6), 
    merge_mode='concat'  # 将正向和反向的输出连接起来
))
regressorBiLSTM.add(Dropout(0.2))

# 添加第二层双向 LSTM 和 Dropout
regressorBiLSTM.add(Bidirectional(LSTM(units=50, return_sequences=True)))
regressorBiLSTM.add(Dropout(0.2))

# 添加第三层双向 LSTM 和 Dropout
regressorBiLSTM.add(Bidirectional(LSTM(units=50, return_sequences=True)))
regressorBiLSTM.add(Dropout(0.2))

# 添加第四层双向 LSTM 和 Dropout
regressorBiLSTM.add(Bidirectional(LSTM(units=50)))
regressorBiLSTM.add(Dropout(0.2))

# 添加输出层
regressorBiLSTM.add(Dense(units=1))  # 输出一个预测值

# 编译 BiLSTM 模型
regressorBiLSTM.compile(optimizer='adam', loss='mean_squared_error')
print(regressorBiLSTM.summary())

# 训练 BiLSTM 模型
history = regressorBiLSTM.fit(
    x_train, y_train, 
    epochs=25, 
    batch_size=32, 
    validation_data=(x_val, y_val), 
    callbacks=[earlystop]  # 仅使用早停回调
)

biend_time = time.time()  # 记录结束时间
bi_runtime = biend_time - bistart_time

# 绘制 BiLSTM 的训练损失和验证损失曲线
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, len(loss) + 1)

plt.figure(figsize=(5, 3))
plt.title('BiLSTM Loss Curve')
plt.plot(epochs_range, loss, 'red', label='Training Loss')
plt.plot(epochs_range, val_loss, 'blue', label='Validation Loss')
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 归一化目标变量 'Close' 列（重复操作，保留以确保功能一致）
sc_one = MinMaxScaler(feature_range=(0, 1))
sc_one.fit(all_data.iloc[:, 1:2])  # 仅拟合，不进行转换

# 预测训练集和测试集的股票价格
predicted_stock_train = regressorBiLSTM.predict(x_train)
predicted_stock_train = sc_one.inverse_transform(predicted_stock_train)

predicted_stock_test_bilstm = regressorBiLSTM.predict(x_test)
predicted_stock_test_bilstm = sc_one.inverse_transform(predicted_stock_test_bilstm)

# 获取真实的股票价格
real_price_train = sc_one.inverse_transform(np.reshape(y_train, (-1, 1)))
real_price_test = sc_one.inverse_transform(np.reshape(y_test, (-1, 1)))

# 可视化 BiLSTM 模型在训练集上的预测结果
plt.figure(figsize=(5, 3))
plt.plot(real_price_train, color='red', label='Real Stock Price')
plt.plot(predicted_stock_train, color='blue', label='Predicted Stock Price')
plt.title('BiLSTM Train Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.grid(True)
plt.legend()
plt.show()

# 可视化 BiLSTM 模型在测试集上的预测结果
plt.figure(figsize=(5, 3))
plt.plot(real_price_test, color='red', label='Real Stock Price')
plt.plot(predicted_stock_test_bilstm, color='blue', label='Predicted Stock Price')
plt.title('BiLSTM Test Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.grid(True)
plt.legend()
plt.show()

# ================================
# 评估 BiLSTM 模型
# ================================

# 计算评估指标
bi_mse = mean_squared_error(predicted_stock_test_bilstm, real_price_test)
bi_rmse = math.sqrt(bi_mse)
bi_mae = mean_absolute_error(predicted_stock_test_bilstm, real_price_test)
bi_r2 = r2_score(predicted_stock_test_bilstm, real_price_test)

# 输出评估结果
print('BiLSTM 模型的均方误差 (MSE): %.6f' % bi_mse)
print('BiLSTM 模型的均方根误差 (RMSE): %.6f' % bi_rmse)
print('BiLSTM 模型的平均绝对误差 (MAE): %.6f' % bi_mae)
print('BiLSTM 模型的决定系数 (R-squared): %.6f' % bi_r2)
print('BiLSTM 模型运行时间: %.2f 秒' % bi_runtime)


# ================================
# 基准模型
# ================================

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# ================================
# 准备数据：转换为二维格式
# ================================
# 将 features 转换为二维格式
features_2d = np.reshape(features, (features.shape[0], -1))

# 划分二维数据集为训练集、验证集和测试集
x_train_2d, x_val_2d, x_test_2d = features_2d[:6550], features_2d[6550:6914], features_2d[6914:]
y_train, y_val, y_test = labels[:6550], labels[6550:6914], labels[6914:]

# ================================
# 构建和训练线性回归模型
# ================================
lr_model = LinearRegression()

lrstart_time = time.time()  # 记录开始时间
lr_model.fit(x_train_2d, y_train)
lrend_time = time.time()  # 记录结束时间

lr_runtime = lrend_time - lrstart_time

# 预测结果
lr_train_pred = lr_model.predict(x_train_2d)
lr_test_pred = lr_model.predict(x_test_2d)

# 反归一化预测值
lr_train_pred_inv = sc_one.inverse_transform(lr_train_pred.reshape(-1, 1))
lr_test_pred_inv = sc_one.inverse_transform(lr_test_pred.reshape(-1, 1))

# 反归一化真实值
real_price_train = sc_one.inverse_transform(np.reshape(y_train, (-1, 1)))
real_price_test = sc_one.inverse_transform(np.reshape(y_test, (-1, 1)))

# 可视化线性回归的测试集预测结果
plt.figure(figsize=(5, 3))
plt.plot(real_price_test, color='red', label='Real Stock Price')
plt.plot(lr_test_pred_inv, color='blue', label='Predicted Stock Price')
plt.title('Linear Regression Test Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.grid(True)
plt.legend()
plt.show()

# ================================
# 构建和训练随机森林模型
# ================================
rf_model = RandomForestRegressor(n_estimators=100, random_state=seed_num)

rfstart_time = time.time()  # 记录开始时间
rf_model.fit(x_train_2d, y_train)
rfend_time = time.time()  # 记录结束时间
rf_runtime = rfend_time - rfstart_time

# 预测结果
rf_train_pred = rf_model.predict(x_train_2d)
rf_test_pred = rf_model.predict(x_test_2d)

# 反归一化预测值
rf_train_pred_inv = sc_one.inverse_transform(rf_train_pred.reshape(-1, 1))
rf_test_pred_inv = sc_one.inverse_transform(rf_test_pred.reshape(-1, 1))

# 可视化随机森林的测试集预测结果
plt.figure(figsize=(5, 3))
plt.plot(real_price_test, color='red', label='Real Stock Price')
plt.plot(rf_test_pred_inv, color='blue', label='Predicted Stock Price')
plt.title('Random Forest Test Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.grid(True)
plt.legend()
plt.show()

# ================================
# 评估线性回归和随机森林模型
# ================================
# 线性回归评估
lr_mse = mean_squared_error(real_price_test, lr_test_pred_inv)
lr_rmse = math.sqrt(lr_mse)
lr_mae = mean_absolute_error(real_price_test, lr_test_pred_inv)
lr_r2 = r2_score(real_price_test, lr_test_pred_inv)

print('线性回归模型的均方误差 (MSE): %.6f' % lr_mse)
print('线性回归模型的均方根误差 (RMSE): %.6f' % lr_rmse)
print('线性回归模型的平均绝对误差 (MAE): %.6f' % lr_mae)
print('线性回归模型的决定系数 (R-squared): %.6f' % lr_r2)
print('线性回归模型运行时间: %.2f 秒' % lr_runtime)

# 随机森林评估
rf_mse = mean_squared_error(real_price_test, rf_test_pred_inv)
rf_rmse = math.sqrt(rf_mse)
rf_mae = mean_absolute_error(real_price_test, rf_test_pred_inv)
rf_r2 = r2_score(real_price_test, rf_test_pred_inv)

print('随机森林模型的均方误差 (MSE): %.6f' % rf_mse)
print('随机森林模型的均方根误差 (RMSE): %.6f' % rf_rmse)
print('随机森林模型的平均绝对误差 (MAE): %.6f' % rf_mae)
print('随机森林模型的决定系数 (R-squared): %.6f' % rf_r2)
print('随机森林模型运行时间: %.2f 秒' % rf_runtime)



# ================================
# 绘制对比图
# ================================

# 模型的评估数据
metrics_values = {
    'LSTM': {'MSE': lstm_mse, 'RMSE': lstm_rmse, 'MAE': lstm_mae, 'R2': lstm_r2, 'Time': lstm_runtime},
    'GRU': {'MSE': gru_mse, 'RMSE': gru_rmse, 'MAE': gru_mae, 'R2': gru_r2, 'Time': gru_runtime},
    'BiLSTM': {'MSE': bi_mse, 'RMSE': bi_rmse, 'MAE': bi_mae, 'R2': bi_r2, 'Time': bi_runtime},
    'LR': {'MSE': lr_mse, 'RMSE': lr_rmse, 'MAE': lr_mae, 'R2': lr_r2, 'Time': lr_runtime},
    'RF': {'MSE': rf_mse, 'RMSE': rf_rmse, 'MAE': rf_mae, 'R2': rf_r2, 'Time': rf_runtime}
}

# 提取数据以绘制图表
metrics = ['MSE', 'RMSE', 'MAE', 'R2', 'Time']
models = list(metrics_values.keys())

# 分别绘制每个指标的柱状图
for metric in metrics:
    metric_scores = [metrics_values[model][metric] for model in models]
    
    plt.figure(figsize=(5, 3))
    plt.bar(models, metric_scores, color='skyblue')
    plt.title(f'{metric} Comparison Across Models')
    plt.xlabel('Models')
    plt.ylabel(metric)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()


In [ ]:
# ================================
# 统计检验：T检验与F检验
# ================================

# 导入额外的库
from scipy.stats import ttest_rel, levene
import matplotlib.pyplot as plt
import seaborn as sns

# ================================
# 计算残差
# ================================

# LSTM 残差
residuals_lstm = real_price_test - predicted_stock_test_lstm

# GRU 残差
residuals_gru = real_price_test - predicted_stock_test_gru

# BiLSTM 残差
residuals_bilstm = real_price_test - predicted_stock_test_bilstm

# 线性回归残差
residuals_lr = real_price_test - lr_test_pred_inv

# 随机森林残差
residuals_rf = real_price_test - rf_test_pred_inv

# ================================
# 执行 T 检验
# ================================

print('===============================')
print('t 检验结果:')
print('===============================')

# 创建列表存储 t 检验结果
t_test_results = []

# LSTM vs GRU
t_stat, p_value = ttest_rel(residuals_lstm.flatten(), residuals_gru.flatten())
print('t 检验（LSTM vs GRU）:')
print('t 统计量 = %.4f, p 值 = %.4f' % (t_stat, p_value))
t_test_results.append({'Model Pair': 'LSTM vs GRU', 't_stat': t_stat, 'p_value': p_value})

# LSTM vs BiLSTM
t_stat, p_value = ttest_rel(residuals_lstm.flatten(), residuals_bilstm.flatten())
print('t 检验（LSTM vs BiLSTM）:')
print('t 统计量 = %.4f, p 值 = %.4f' % (t_stat, p_value))
t_test_results.append({'Model Pair': 'LSTM vs BiLSTM', 't_stat': t_stat, 'p_value': p_value})

# LSTM vs Linear Regression
t_stat, p_value = ttest_rel(residuals_lstm.flatten(), residuals_lr.flatten())
print('t 检验（LSTM vs Linear Regression）:')
print('t 统计量 = %.4f, p 值 = %.4f' % (t_stat, p_value))
t_test_results.append({'Model Pair': 'LSTM vs Linear Regression', 't_stat': t_stat, 'p_value': p_value})

# LSTM vs Random Forest
t_stat, p_value = ttest_rel(residuals_lstm.flatten(), residuals_rf.flatten())
print('t 检验（LSTM vs Random Forest）:')
print('t 统计量 = %.4f, p 值 = %.4f' % (t_stat, p_value))
t_test_results.append({'Model Pair': 'LSTM vs Random Forest', 't_stat': t_stat, 'p_value': p_value})

# GRU vs BiLSTM
t_stat, p_value = ttest_rel(residuals_gru.flatten(), residuals_bilstm.flatten())
print('t 检验（GRU vs BiLSTM）:')
print('t 统计量 = %.4f, p 值 = %.4f' % (t_stat, p_value))
t_test_results.append({'Model Pair': 'GRU vs BiLSTM', 't_stat': t_stat, 'p_value': p_value})

# Linear Regression vs Random Forest
t_stat, p_value = ttest_rel(residuals_lr.flatten(), residuals_rf.flatten())
print('t 检验（Linear Regression vs Random Forest）:')
print('t 统计量 = %.4f, p 值 = %.4f' % (t_stat, p_value))
t_test_results.append({'Model Pair': 'Linear Regression vs Random Forest', 't_stat': t_stat, 'p_value': p_value})

# 转换为 DataFrame
t_test_df = pd.DataFrame(t_test_results)


# ================================
# 执行 F 检验（Levene’s Test）
# ================================

print('===============================')
print('F 检验（Levene’s Test）结果:')
print('===============================')

# 创建列表存储 F 检验结果
f_test_results = []

# LSTM vs GRU
f_stat, p_value = levene(residuals_lstm.flatten(), residuals_gru.flatten())
print('F 检验（LSTM vs GRU）:')
print('F 统计量 = %.4f, p 值 = %.4f' % (f_stat, p_value))
f_test_results.append({'Model Pair': 'LSTM vs GRU', 'F_stat': f_stat, 'p_value': p_value})

# LSTM vs BiLSTM
f_stat, p_value = levene(residuals_lstm.flatten(), residuals_bilstm.flatten())
print('F 检验（LSTM vs BiLSTM）:')
print('F 统计量 = %.4f, p 值 = %.4f' % (f_stat, p_value))
f_test_results.append({'Model Pair': 'LSTM vs BiLSTM', 'F_stat': f_stat, 'p_value': p_value})

# LSTM vs Linear Regression
f_stat, p_value = levene(residuals_lstm.flatten(), residuals_lr.flatten())
print('F 检验（LSTM vs Linear Regression）:')
print('F 统计量 = %.4f, p 值 = %.4f' % (f_stat, p_value))
f_test_results.append({'Model Pair': 'LSTM vs Linear Regression', 'F_stat': f_stat, 'p_value': p_value})

# LSTM vs Random Forest
f_stat, p_value = levene(residuals_lstm.flatten(), residuals_rf.flatten())
print('F 检验（LSTM vs Random Forest）:')
print('F 统计量 = %.4f, p 值 = %.4f' % (f_stat, p_value))
f_test_results.append({'Model Pair': 'LSTM vs Random Forest', 'F_stat': f_stat, 'p_value': p_value})

# GRU vs BiLSTM
f_stat, p_value = levene(residuals_gru.flatten(), residuals_bilstm.flatten())
print('F 检验（GRU vs BiLSTM）:')
print('F 统计量 = %.4f, p 值 = %.4f' % (f_stat, p_value))
f_test_results.append({'Model Pair': 'GRU vs BiLSTM', 'F_stat': f_stat, 'p_value': p_value})

# Linear Regression vs Random Forest
f_stat, p_value = levene(residuals_lr.flatten(), residuals_rf.flatten())
print('F 检验（Linear Regression vs Random Forest）:')
print('F 统计量 = %.4f, p 值 = %.4f' % (f_stat, p_value))
f_test_results.append({'Model Pair': 'Linear Regression vs Random Forest', 'F_stat': f_stat, 'p_value': p_value})

# 转换为 DataFrame
f_test_df = pd.DataFrame(f_test_results)


# ================================
# 可视化残差分布
# ================================

# 创建一个 DataFrame 包含所有残差
residuals_df = pd.DataFrame({
    'LSTM': residuals_lstm.flatten(),
    'GRU': residuals_gru.flatten(),
    'BiLSTM': residuals_bilstm.flatten(),
    'Linear Regression': residuals_lr.flatten(),
    'Random Forest': residuals_rf.flatten()
})

# 绘制箱线图比较各模型的残差分布
plt.figure(figsize=(12, 6))
sns.boxplot(data=residuals_df, palette='Set3')
plt.title('Box Plot of Residual Distributions for Different Models')
plt.xlabel('Models')
plt.ylabel('Residual Distributions')
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.show()

# 绘制小提琴图比较各模型的残差分布
plt.figure(figsize=(12, 6))
sns.violinplot(data=residuals_df, palette='Set2', inner='quartile')
plt.title('Violin Plot of Residual Distributions for Different Models')
plt.xlabel('Models')
plt.ylabel('Residual Distributions')
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.show()

# ================================
# Cohen's d 效应量比较所有模型的残差
# ================================


# 定义 Cohen's d 函数
def cohen_d(x, y):
    diff = x - y
    return np.mean(diff) / np.std(diff, ddof=1)

# 计算所有模型对之间的效应量
d_lstm_gru = cohen_d(residuals_lstm.flatten(), residuals_gru.flatten())
d_lstm_bilstm = cohen_d(residuals_lstm.flatten(), residuals_bilstm.flatten())
d_lstm_lr = cohen_d(residuals_lstm.flatten(), residuals_lr.flatten())
d_lstm_rf = cohen_d(residuals_lstm.flatten(), residuals_rf.flatten())
d_gru_bilstm = cohen_d(residuals_gru.flatten(), residuals_bilstm.flatten())
d_lr_rf = cohen_d(residuals_lr.flatten(), residuals_rf.flatten())

# 打印结果
print(f"Cohen's d for LSTM vs GRU: {d_lstm_gru:.4f}")
print(f"Cohen's d for LSTM vs BiLSTM: {d_lstm_bilstm:.4f}")
print(f"Cohen's d for LSTM vs Linear Regression: {d_lstm_lr:.4f}")
print(f"Cohen's d for LSTM vs Random Forest: {d_lstm_rf:.4f}")
print(f"Cohen's d for GRU vs BiLSTM: {d_gru_bilstm:.4f}")
print(f"Cohen's d for Linear Regression vs Random Forest: {d_lr_rf:.4f}")

# 汇总 Cohen's d 结果
models = [
    "LSTM vs GRU",
    "LSTM vs BiLSTM",
    "LSTM vs Linear Regression",
    "LSTM vs Random Forest",
    "GRU vs BiLSTM",
    "Linear Regression vs Random Forest"
]
cohen_d_values = [d_lstm_gru, d_lstm_bilstm, d_lstm_lr, d_lstm_rf, d_gru_bilstm, d_lr_rf]

# 绘制柱状图
plt.figure(figsize=(10, 6))
plt.bar(models, cohen_d_values, color='skyblue')
plt.title("Cohen's d Effect Sizes Between Models")
plt.xlabel("Model Comparisons")
plt.ylabel("Effect Size (Cohen's d)")
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# ================================
# 模型的残差绘制 QQ 图
# ================================

from scipy.stats import probplot

# 创建模型残差的字典，方便迭代处理
model_residuals = {
    "LSTM": residuals_lstm.flatten(),
    "GRU": residuals_gru.flatten(),
    "BiLSTM": residuals_bilstm.flatten(),
    "LR": residuals_lr.flatten(),
    "RF": residuals_rf.flatten()
}

# 设置图形大小和布局
plt.figure(figsize=(15, 10))
n_models = len(model_residuals)
rows = (n_models + 2) // 3  # 每行最多放 3 个子图

# 遍历模型残差，逐个绘制 QQ 图
for i, (model_name, residuals) in enumerate(model_residuals.items(), 1):
    plt.subplot(rows, 3, i)  # 创建子图
    probplot(residuals, dist="norm", plot=plt)
    plt.title(f'QQ Plot of {model_name} Residuals')

# 调整布局并显示
plt.tight_layout()
plt.show()


In [ ]:
# ================================
# 可解释性分析 SHAP 
# 涉及模型：LSTM / GRU / BiLSTM / 线性回归 / 随机森林
# ================================

import shap  # 需要关注内核与版本的兼容问题
import numpy as np
import matplotlib.pyplot as plt

# ---------------------------------------------------
# 第 0 步：对模型输入做“二维化”处理用于 SHAP
# ---------------------------------------------------

# 主代码中，features 的形状为 (N, 60, 6)
# 需要将其拉伸成 (N, 360)使得 KernelExplainer 能解析
features_2d = features.reshape(features.shape[0], -1)  # (N, 360)
print("Flattened features shape:", features_2d.shape)

# 数据集切分 (x_train, x_val, x_test ...) 保持与原训练一致
# 拆分：前 6550 用作训练，6550-6914 为验证，6914 之后为测试
x_train_2d = features_2d[:6550]
x_val_2d   = features_2d[6550:6914]
x_test_2d  = features_2d[6914:]

y_train = labels[:6550]
y_val   = labels[6550:6914]
y_test  = labels[6914:]

# ---------------------------------------------------
# 第 1 步：定义各模型预测函数 (LSTM / GRU / BiLSTM / 线性回归)
# ---------------------------------------------------
def lstm_predict_function(x_2d):
    """
    2D 输入 (samples, 360) 还原为 3D (samples, 60, 6)，
    然后调用训练好的 LSTM 模型预测，并返回 1D 数组 preds。
    """
    x_3d = x_2d.reshape((x_2d.shape[0], 60, 6))
    preds = regressor.predict(x_3d).flatten()  # 使用 flatten() 保持为一维数组
    return preds

def gru_predict_function(x_2d):
    """
    2D 输入还原为 3D，调用训练好的 GRU 模型预测，并返回 1D 数组 preds。
    """
    x_3d = x_2d.reshape((x_2d.shape[0], 60, 6))
    preds = regressorGRU.predict(x_3d).flatten()
    return preds

def bilstm_predict_function(x_2d):
    """
    2D 输入还原为 3D，调用训练好的 BiLSTM 模型预测，并返回 1D 数组 preds。
    """
    x_3d = x_2d.reshape((x_2d.shape[0], 60, 6))
    preds = regressorBiLSTM.predict(x_3d).flatten()
    return preds

def linear_regression_predict_function(x_2d):
    """
    直接使用训练好的线性回归模型进行预测，并返回 1D 数组 preds。
    """
    preds = lr_model.predict(x_2d).flatten()
    return preds

# ---------------------------------------------------
# 随机森林是二维输入，直接用 TreeExplainer
# ---------------------------------------------------
# rf_model 为已训练好的 RandomForestRegressor




# ---------------------------------------------------
# 第 2 步：准备背景数据 (Background Data)
# ---------------------------------------------------
# KernelExplainer 需要一批数据作为背景分布来近似计算Shapley值
# background_data = x_train_2d[:500] 一天LSTM进度只有28%需要改变思路，不然来不及
background_data = shap.sample(x_train_2d, 60)  # 60 耗费12小时完成


# ---------------------------------------------------
# 第 3 步：创建各模型的 explainer
# ---------------------------------------------------
# 深度学习模型（LSTM / GRU / BiLSTM），使用 KernelExplainer
# 线性模型（线性回归），使用 LinearExplainer
# 树的模型（随机森林），使用 TreeExplainer

# LSTM 的 explainer
explainer_lstm = shap.KernelExplainer(lstm_predict_function, data=background_data, link='identity')

# GRU 的 explainer
explainer_gru = shap.KernelExplainer(gru_predict_function, data=background_data, link='identity')

# BiLSTM 的 explainer
explainer_bilstm = shap.KernelExplainer(bilstm_predict_function, data=background_data, link='identity')

# 线性回归 的 explainer
explainer_lr = shap.LinearExplainer(lr_model, x_train_2d, feature_perturbation="interventional")

# 随机森林 的 explainer
explainer_rf = shap.TreeExplainer(rf_model)

# ---------------------------------------------------
# 第 4 步：选择要解释的测试集样本
# ---------------------------------------------------
num_explain = 60  # 因为时间目前只解释前 60 条测试样本
to_explain_2d = x_test_2d[:num_explain]



# ---------------------------------------------------
# 第 5 步：计算 SHAP 值
# ---------------------------------------------------
print("Computing SHAP values for LSTM (KernelExplainer) ...")
shap_values_lstm = explainer_lstm.shap_values(to_explain_2d)

print("Computing SHAP values for GRU (KernelExplainer) ...")
shap_values_gru = explainer_gru.shap_values(to_explain_2d)

print("Computing SHAP values for BiLSTM (KernelExplainer) ...")
shap_values_bilstm = explainer_bilstm.shap_values(to_explain_2d)

print("Computing SHAP values for Linear Regression (LinearExplainer) ...")
shap_values_lr = explainer_lr.shap_values(to_explain_2d)

print("Computing SHAP values for Random Forest (TreeExplainer) ...")
shap_values_rf = explainer_rf.shap_values(to_explain_2d)

# ---------------------------------------------------
# 第 6 步：特征命名方便批量区分
# ---------------------------------------------------
feature_names = []
time_steps = 60
feat_list = ["Open", "Close", "High", "Low", "Volume", "AdjClose"]
for t in range(time_steps):
    for f in feat_list:
        feature_names.append(f"t{t}_{f}")

        
        
# ---------------------------------------------------
# 第 7 步：可视化 
# ---------------------------------------------------

print("\nSHAP Summary Plot for LSTM:")
shap.summary_plot(
    shap_values_lstm, 
    to_explain_2d, 
    feature_names=feature_names, 
    plot_type='bar'  # 此处 'bar' 比 'dot' 更直观
)

# GRU
print("\nSHAP Summary Plot for GRU:")
shap.summary_plot(
    shap_values_gru, 
    to_explain_2d, 
    feature_names=feature_names, 
    plot_type='bar'
)

# BiLSTM
print("\nSHAP Summary Plot for BiLSTM:")
shap.summary_plot(
    shap_values_bilstm, 
    to_explain_2d, 
    feature_names=feature_names, 
    plot_type='bar'
)

# 线性回归
print("\nSHAP Summary Plot for Linear Regression:")
shap.summary_plot(
    shap_values_lr, 
    to_explain_2d, 
    feature_names=feature_names, 
    plot_type='bar'
)

# 随机森林 
print("\nSHAP Summary Plot for Random Forest:")
shap.summary_plot(
    shap_values_rf, 
    x_test_2d[:num_explain], 
    feature_names=feature_names, 
    plot_type='bar'
)

# ---------------------------------------------------
# 第 8 步：可视化 (force_plot) - 单样本局部解释 
# ---------------------------------------------------
sample_idx = 0  # 选择第一个样本

print(f"\nSHAP Force Plot for Sample Index: {sample_idx}")

# LSTM force plot
print("LSTM Force Plot:")
shap.force_plot(
    explainer_lstm.expected_value, 
    shap_values_lstm[sample_idx, :], 
    to_explain_2d[sample_idx, :],
    feature_names=feature_names,
    matplotlib=True
)
plt.show()

# GRU force plot
print("GRU Force Plot:")
shap.force_plot(
    explainer_gru.expected_value, 
    shap_values_gru[sample_idx, :], 
    to_explain_2d[sample_idx, :],
    feature_names=feature_names,
    matplotlib=True
)
plt.show()

# BiLSTM force plot
print("BiLSTM Force Plot:")
shap.force_plot(
    explainer_bilstm.expected_value, 
    shap_values_bilstm[sample_idx, :], 
    to_explain_2d[sample_idx, :],
    feature_names=feature_names,
    matplotlib=True
)
plt.show()

# 线性回归 force plot
print("Linear Regression Force Plot:")
shap.force_plot(
    explainer_lr.expected_value, 
    shap_values_lr[sample_idx, :], 
    to_explain_2d[sample_idx, :],
    feature_names=feature_names,
    matplotlib=True
)
plt.show()

# 随机森林 force plot
print("Random Forest Force Plot:")
shap.force_plot(
    explainer_rf.expected_value, 
    shap_values_rf[sample_idx, :], 
    x_test_2d[sample_idx, :],
    feature_names=feature_names,
    matplotlib=True
)
plt.show()

# ---------------------------------------------------
# 第 9 步：可视化“时间步 vs. 特征” SHAP 值 的深度分析
# ---------------------------------------------------
def visualize_time_step_shap(shap_value_1d, sample_idx=0):
    """
    shap_value_1d: 单个样本的 shap 值向量 (长度=360)，来自 shap_values_lstm[sample_idx,:] 等
    sample_idx: 用于打印提示信息
    """
    # 还原为 (60, 6)
    shap_2d = shap_value_1d.reshape(60, 6)
    plt.figure(figsize=(10, 6))
    plt.imshow(shap_2d, cmap='RdBu', aspect='auto')
    plt.colorbar(label='SHAP Value')
    plt.xlabel("Feature Index (0:Open,1:Close,2:High,3:Low,4:Volume,5:AdjClose)")
    plt.ylabel("Time Step (0 ~ 59)")
    plt.title(f"Sample {sample_idx} TimeStep-Feature SHAP Heatmap")
    plt.show()

# 可视化第0条样本的各模型 SHAP 值
print("\nSHAP Heatmap for LSTM:")
visualize_time_step_shap(shap_values_lstm[sample_idx, :], sample_idx=sample_idx)

print("\nSHAP Heatmap for GRU:")
visualize_time_step_shap(shap_values_gru[sample_idx, :], sample_idx=sample_idx)

print("\nSHAP Heatmap for BiLSTM:")
visualize_time_step_shap(shap_values_bilstm[sample_idx, :], sample_idx=sample_idx)

print("\nSHAP Heatmap for Linear Regression:")
visualize_time_step_shap(shap_values_lr[sample_idx, :], sample_idx=sample_idx)

# 随机森林的特征是二维的可以直接可视化
def visualize_tree_shap(shap_values, sample_idx=0):
    """
    shap_values: 随机森林模型的 shap 值
    sample_idx: 样本索引
    """
    shap_1d = shap_values[sample_idx, :]
    shap_2d = shap_1d.reshape(60, 6)
    plt.figure(figsize=(10, 6))
    plt.imshow(shap_2d, cmap='RdBu', aspect='auto')
    plt.colorbar(label='SHAP Value')
    plt.xlabel("Feature Index (0:Open,1:Close,2:High,3:Low,4:Volume,5:AdjClose)")
    plt.ylabel("Time Step (0 ~ 59)")
    plt.title(f"Sample {sample_idx} TimeStep-Feature SHAP Heatmap for Random Forest")
    plt.show()

print("\nSHAP Heatmap for Random Forest:")
visualize_tree_shap(shap_values_rf, sample_idx=sample_idx)

print("全部SHAP分析完成")
